In [59]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import re
from scipy import stats

# Load Dataset

In [60]:
df = pd.read_csv("Salinan Online Retail Data.csv")
print(df.shape)
print(df.info())
df.head(10)

(461773, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB
None


,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
5,493413,21723,ALPHABET HEARTS STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
6,493414,21844,RETRO SPOT MUG,36,2010-01-04 10:28:00,2.55,14590.0
7,493414,21533,RETRO SPOT LARGE MILK JUG,12,2010-01-04 10:28:00,4.25,14590.0
8,493414,37508,NEW ENGLAND CERAMIC CAKE SERVER,2,2010-01-04 10:28:00,2.55,14590.0
9,493414,35001G,HAND OPEN SHAPE GOLD,2,2010-01-04 10:28:00,4.25,14590.0


# Data Cleansing

In [61]:
# Drop N/A values
df.dropna(how='any',inplace=True)

# Remove Unnecessary rows
df = df[~df['order_id'].str.contains('C')]
df = df[(~df['product_code'].str.contains('TEST')) | (~df['product_name'].str.contains('test'))]
df = df[df['price'] > 0]

# Remove Outlier
df = df[(np.abs(stats.zscore(df[['quantity','price']])) < 3).all(axis=1)]

# Change Dtype
df['order_date'] = df['order_date'].astype('datetime64[ns]')
df['customer_id'] = df['customer_id'].astype('str')

# Fixing different product_name on the same product_code issue
temp = df.groupby(['product_code','product_name'],as_index=False).agg(count=('order_id','nunique')).sort_values(['product_code','count'],ascending=[True,False])
temp['rank'] = temp.groupby('product_code')['count'].rank(method='first',ascending=False).astype('int')
temp = temp[temp['rank'] == 1].drop(columns=['rank','count'])
temp.rename(columns={'product_name':'frequent_product_name'},inplace=True)
df = df.merge(temp,how='left',on='product_code')
df['product_name'] = df['frequent_product_name']
df.drop('frequent_product_name',axis=1,inplace=True)

df.reset_index(drop=True,inplace=True)
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493414,21844,RED RETROSPOT MUG,36,2010-01-04 10:28:00,2.55,14590.0
1,493414,21533,RETRO SPOT LARGE MILK JUG,12,2010-01-04 10:28:00,4.25,14590.0
2,493414,37508,NEW ENGLAND CERAMIC CAKE SERVER,2,2010-01-04 10:28:00,2.55,14590.0
3,493414,35001G,HAND OPEN SHAPE GOLD,2,2010-01-04 10:28:00,4.25,14590.0
4,493414,21527,RED RETROSPOT TRADITIONAL TEAPOT,12,2010-01-04 10:28:00,6.95,14590.0
...,...,...,...,...,...,...,...
351278,539988,84380,SET OF 3 BUTTERFLY COOKIE CUTTERS,1,2010-12-23 16:06:00,1.25,18116.0
351279,539988,84849D,HOT BATHS SOAP HOLDER,1,2010-12-23 16:06:00,1.69,18116.0
351280,539988,84849B,FAIRY SOAP SOAP HOLDER,1,2010-12-23 16:06:00,1.69,18116.0
351281,539988,22854,CREAM SWEETHEART EGG HOLDER,2,2010-12-23 16:06:00,4.95,18116.0


In [62]:
print(df.info())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351283 entries, 0 to 351282
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      351283 non-null  object        
 1   product_code  351283 non-null  object        
 2   product_name  351283 non-null  object        
 3   quantity      351283 non-null  int64         
 4   order_date    351283 non-null  datetime64[ns]
 5   price         351283 non-null  float64       
 6   customer_id   351283 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 18.8+ MB
None


,quantity,order_date,price
count,351283.000000,351283,351283.000000
mean,9.636723,2010-07-22 13:27:39.280636928,2.969548
min,1.000000,2010-01-04 10:28:00,0.001000
25%,2.000000,2010-04-25 12:34:00,1.250000
50%,4.000000,2010-08-03 12:13:00,1.950000
75%,12.000000,2010-10-25 10:41:00,3.750000
max,212.000000,2010-12-23 16:06:00,95.000000
std,17.129233,NaN,3.236739


# Create Basket

In [70]:
basket = df.groupby('order_id')['product_name'].apply(list).reset_index()
basket_encoded = basket['product_name'].str.join('|').str.get_dummies('|').astype('bool')
basket_encoded

,10 COLOUR SPACEBOY PEN,12 ASS ZINC CHRISTMAS DECORATIONS,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 MINI TOADSTOOL PEGS,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE POSY,...,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE DOUBLE PLANTER,ZINC HEART LATTICE PLANTER BOWL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC METAL HEART DECORATION,ZINC POLICE BOX LANTERN,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16467,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16468,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16469,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16470,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Apriori Algorithm

In [71]:
frequent_itemset = apriori(basket_encoded,min_support=.01,use_colnames=True).sort_values('support',ascending=False).reset_index(drop=True)
frequent_itemset['product_count'] = frequent_itemset['itemsets'].apply(len)

,support,itemsets
0,0.165736,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.094828,(REGENCY CAKESTAND 3 TIER)
2,0.090517,(JUMBO BAG RED RETROSPOT)
3,0.072851,(PACK OF 72 RETRO SPOT CAKE CASES)
4,0.072062,(ASSORTED COLOUR BIRD ORNAMENT)
...,...,...
1006,0.010017,"(72 SWEETHEART FAIRY CAKE CASES, WHITE HANGING..."
1007,0.010017,"(JUMBO BAG OWLS, JUMBO STORAGE BAG SUKI, JUMBO..."
1008,0.010017,(CHRISTMAS GINGHAM STAR)
1009,0.010017,"(PLASTERS IN TIN VINTAGE PAISLEY, PLASTERS IN ..."


In [74]:
frequent_itemset

,support,itemsets,product_count
0,0.165736,(WHITE HANGING HEART T-LIGHT HOLDER),1
1,0.094828,(REGENCY CAKESTAND 3 TIER),1
2,0.090517,(JUMBO BAG RED RETROSPOT),1
3,0.072851,(PACK OF 72 RETRO SPOT CAKE CASES),1
4,0.072062,(ASSORTED COLOUR BIRD ORNAMENT),1
...,...,...,...
1006,0.010017,"(72 SWEETHEART FAIRY CAKE CASES, WHITE HANGING...",2
1007,0.010017,"(JUMBO BAG OWLS, JUMBO STORAGE BAG SUKI, JUMBO...",3
1008,0.010017,(CHRISTMAS GINGHAM STAR),1
1009,0.010017,"(PLASTERS IN TIN VINTAGE PAISLEY, PLASTERS IN ...",2


In [75]:
product_association = association_rules(frequent_itemset,metric='confidence',min_threshold=.7).sort_values(['support','confidence'],ascending=[False,False]).reset_index(drop=True)
product_association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(RED HANGING HEART T-LIGHT HOLDER),(WHITE HANGING HEART T-LIGHT HOLDER),0.054031,0.165736,0.039157,0.724719,4.372737,0.030202,3.030592,0.815365
1,(SWEETHEART CERAMIC TRINKET BOX),(STRAWBERRY CERAMIC TRINKET BOX),0.044985,0.068419,0.034119,0.758435,11.085123,0.031041,3.856433,0.952644
2,(TOILET METAL SIGN),(BATHROOM METAL SIGN),0.024587,0.036972,0.019791,0.804938,21.771664,0.018882,4.937043,0.978118
3,(PAINTED METAL PEARS ASSORTED),(ASSORTED COLOUR BIRD ORNAMENT),0.020398,0.072062,0.015420,0.755952,10.490352,0.013950,3.802284,0.923512
4,(RED RETROSPOT SUGAR JAM BOWL),(RED RETROSPOT SMALL MILK JUG),0.021612,0.033936,0.015299,0.707865,20.858596,0.014565,3.306910,0.973089
...,...,...,...,...,...,...,...,...,...,...
59,(CHILDS GARDEN TROWEL PINK),(CHILDS GARDEN FORK PINK),0.012749,0.011231,0.010442,0.819048,72.926229,0.010299,5.464249,0.999024
60,"(CHARLOTTE BAG PINK WITH WHITE SPOTS, STRAWBER...",(RED SPOTTY CHARLOTTE BAG),0.013052,0.036425,0.010321,0.790698,21.707287,0.009845,4.603745,0.966548
61,"(KEY FOB , FRONT DOOR )","(KEY FOB , SHED)",0.014085,0.022827,0.010138,0.719828,31.534574,0.009817,3.487757,0.982121
62,"(CHARLOTTE BAG PINK WITH WHITE SPOTS, CHARLOTT...",(RED SPOTTY CHARLOTTE BAG),0.014206,0.036425,0.010078,0.709402,19.475442,0.009560,3.315830,0.962324
